In [1]:
import pandas as pd
import os
import string
import re

In [2]:
# Read and process movies table 

In [3]:
movies_df = pd.read_csv('movies.csv')

In [4]:
# Make new column with release year
movies_df['year'] = movies_df['title'].str.extract(r'.*\((\d\d\d\d)\).*')  # Regex to capture year 
movies_df.dropna(inplace=True)  # Drop movies that don't have a year in the title
movies_df['year'] = movies_df['year'].astype(int)

movies_df['title'] = movies_df['title'].str.replace(r'\((\d\d\d\d)\)', '')  # Remove year from title
movies_df['title'] = movies_df['title'].str.replace('[{}]'.format(string.punctuation), '')  # Remove ponctuation from title
movies_df['title'] = movies_df['title'].str.replace(' ', '')
    
movies_df['genres'] = movies_df['genres'].str.replace('|', '/')  # Remove year from title
movies_df['genres'] = movies_df['genres'].str.replace('\(no genres listed\)', 'NA')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


In [5]:
print(movies_df.shape)
movies_df.head()

(27256, 4)


,movieId,title,genres,year
0,1,ToyStory,Adventure/Animation/Children/Comedy/Fantasy,1995
1,2,Jumanji,Adventure/Children/Fantasy,1995
2,3,GrumpierOldMen,Comedy/Romance,1995
3,4,WaitingtoExhale,Comedy/Drama/Romance,1995
4,5,FatheroftheBridePartII,Comedy,1995


# Define filters

In [6]:
def filter_60s_through_80s(df):
    '''60sto80s'''
    return df[df['year'].between(1960, 1990)]

# filter_60s_through_80s(movies_df)  # 6k
# filter_60s_through_80s.__doc__

In [7]:
def filter_90s(df):
    '''90s'''
    return df[df['year'].between(1990, 2000 )]

# filter_90s(movies_df)  # 17k

In [8]:
def filter_00s_through_now(df):
    '''00stonow'''
    return df[df['year'].between(2000, 2018)]

# filter_00s_through_now(movies_df)  # 12k

In [9]:
def filter_children(df):
    '''children'''
    return df[df['genres'].str.contains('Children', regex=False)]

# filter_children(movies_df)  # 1k

In [10]:
def filter_action(df):
    '''action'''
    return df[df['genres'].str.contains('Action', regex=False)]

# filter_action(movies_df)  # 3k

In [11]:
def filter_documentaries(df):
    '''documentaries'''
    return df[df['genres'].str.contains('Documentary', regex=False)]

# filter_documentaries(movies_df)  # 2k

In [12]:
def filter_no_genre(df):
    '''nogenre'''
    return df[df['genres'].str.contains('NA', regex=False)]

# filter_no_genre(movies_df)  # 237

In [13]:
def filter_random_2000(df):
    '''2krand'''
    return df.sample(n=2000, random_state=42)

# filter_random_2000(movies_df)

In [14]:
def filter_4_or_more_genres(df):
    '''4plusgenres'''
    return df[df['genres'].str.count('/') >= 3]

# filter_3_or_more_genres(movies_df)  # 2k

In [15]:
filter_list = [filter_60s_through_80s, filter_90s, filter_00s_through_now, filter_children, 
               filter_action, filter_documentaries, filter_no_genre, filter_random_2000, filter_4_or_more_genres]
for f in filter_list:
    print(f.__doc__, f(movies_df).shape)

60sto80s (6649, 4)
90s (4985, 4)
00stonow (12746, 4)
children (1139, 4)
action (3519, 4)
documentaries (2467, 4)
nogenre (237, 4)
2krand (2000, 4)
4plusgenres (2309, 4)


In [16]:
# Read ratings
ratings_df = pd.read_csv('ratings.csv')
ratings_df['timestamp'] = pd.to_datetime(ratings_df['timestamp'], unit='s')  # Convert unix timestamp to pandas time
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


# Generate datasets with yearly aggregation

In [17]:
yearly_gb = ratings_df.groupby(pd.Grouper(key='timestamp', freq='Y'))

In [18]:
%%time
for filter_function in filter_list:  # Iterate over all filters
    i = 0
    for name, group in yearly_gb:
        # Filter movies
        filtered_movies = filter_function(movies_df)
        stats = group[group['movieId'].isin(filtered_movies['movieId'])].groupby(by='movieId').describe()
        if 'rating' not in stats:  # Check if there are items in filter, else skip month/year
            continue

        # Merge all info from statistics and movies into a single df
        stats = stats['rating'][['count', 'mean', 'std']]
        df_final = stats.merge(filtered_movies[['movieId', 'title', 'genres', 'year']], left_index=True, right_on='movieId', how='inner')

        # Generate correct ids
        df_final['id_genres'] = df_final['genres'] + '/' + df_final['title']
        df_final['id_release'] = df_final['year'].astype(str) + '/' + df_final['title']

        # for a given filter, generate 6 datasets varying weight_col and hierarchy mode 
        for weight_col in ['count', 'mean', 'std']:
            for hierarchy in ['id_genres', 'id_release']:
                # tmdb-yearly-count-genres-action
                dataset_id = 'yearly' + '-' + weight_col + '-' + re.match(r'id_(.*)', hierarchy, re.M|re.I).group(1) + '-' + filter_function.__doc__

                copy = df_final[[hierarchy, weight_col]].dropna(axis=0).copy()  # Get only target cols            
                copy.columns = ['id', 'weight']
                copy = copy[copy['weight'] != 0]  # Remove cols with 0

                os.makedirs('tmbd-datasets/tmbd-' + dataset_id, exist_ok=True)
                copy.to_csv('tmbd-datasets/tmbd-' + dataset_id + '/' + str(name.date()) + '.csv', index=False)
        
        print(str(name.date()) + '    ', end='\r', flush=True)
                
#         i += 1
#         if i == 3:
#             break

CPU times: user 21min 55s, sys: 1.19 s, total: 21min 56s
Wall time: 21min 56s


# Generate datasets with monthly aggregation

In [19]:
monthly_gb = ratings_df.groupby(pd.Grouper(key='timestamp', freq='M'))

In [20]:
%%time
for filter_function in filter_list:  # Iterate over all filters
    i = 0
    for name, group in monthly_gb:
        # Filter movies
        filtered_movies = filter_function(movies_df)
        stats = group[group['movieId'].isin(filtered_movies['movieId'])].groupby(by='movieId').describe()
        if 'rating' not in stats:  # Check if there are items in filter, else skip month/year
            continue

        # Merge all info from statistics and movies into a single df
        stats = stats['rating'][['count', 'mean', 'std']]
        df_final = stats.merge(filtered_movies[['movieId', 'title', 'genres', 'year']], left_index=True, right_on='movieId', how='inner')

        # Generate correct ids
        df_final['id_genres'] = df_final['genres'] + '/' + df_final['title']
        df_final['id_release'] = df_final['year'].astype(str) + '/' + df_final['title']

        # for a given filter, generate 6 datasets varying weight_col and hierarchy mode 
        for weight_col in ['count', 'mean', 'std']:
            for hierarchy in ['id_genres', 'id_release']:
                # tmdb-monthly-count-genres-action
                dataset_id = 'monthly' + '-' + weight_col + '-' + re.match(r'id_(.*)', hierarchy, re.M|re.I).group(1) + '-' + filter_function.__doc__

                copy = df_final[[hierarchy, weight_col]].dropna(axis=0).copy()  # Get only target cols            
                copy.columns = ['id', 'weight']
                copy = copy[copy['weight'] != 0]  # Remove cols with 0

                os.makedirs('tmbd-datasets/tmbd-' + dataset_id, exist_ok=True)
                copy.to_csv('tmbd-datasets/tmbd-' + dataset_id + '/' + str(name.date()) + '.csv', index=False)

        print(str(name.date()) + '    ', end='\r', flush=True)

#         i += 1
#         if i == 3:
#             break

CPU times: user 2h 26min 2s, sys: 3.46 s, total: 2h 26min 6s
Wall time: 2h 26min 2s
